<a href="https://colab.research.google.com/github/aayushkhadka303/AaYushk/blob/main/021-303%20AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Reflex Based Ai agent
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import random
from IPython.display import HTML

GRID_SIZE    = 7
INTERVAL_MS  = 200
NUM_EPISODES = 3

COLORS = {
    'dirty':   (1.00, 0.39, 0.39),  # Red
    'clean':   (1.00, 1.00, 1.00),  # White
    'visited': (0.39, 0.78, 0.39),  # Green
    'agent':   (0.39, 0.39, 1.00),  # Blue
}

class RandomWalkVacuumAgent:
    def __init__(self, size):
        self.size = size

    def reset(self):
        self.grid = np.random.randint(0, 2, (self.size, self.size))
        self.pos = (random.randrange(self.size), random.randrange(self.size))
        self.visited = {self.pos}

    def neighbors(self, x, y):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                yield (nx, ny)

    def step(self):
        x, y = self.pos
        if self.grid[x, y] == 0:
            self.grid[x, y] = 1
        else:
            adj = list(self.neighbors(x, y))
            dirty_adj = [p for p in adj if self.grid[p] == 0]
            if dirty_adj:
                nxt = random.choice(dirty_adj)
            else:
                unvisited_adj = [p for p in adj if p not in self.visited]
                if unvisited_adj:
                    nxt = random.choice(unvisited_adj)
                else:
                    nxt = random.choice(adj)
            self.pos = nxt
            self.visited.add(nxt)
        return True

    def run_episode(self):
        frames = []
        total_cells = self.size * self.size
        steps = 0
        while not (np.all(self.grid == 1) and len(self.visited) == total_cells):
            frame = np.zeros((self.size, self.size, 3))
            for i in range(self.size):
                for j in range(self.size):
                    if (i, j) == self.pos:
                        frame[i, j] = COLORS['agent']
                    elif self.grid[i, j] == 0:
                        frame[i, j] = COLORS['dirty']
                    elif (i, j) in self.visited:
                        frame[i, j] = COLORS['visited']
                    else:
                        frame[i, j] = COLORS['clean']
            frames.append(frame)
            self.step()
            steps += 1
        return frames, steps

agent = RandomWalkVacuumAgent(GRID_SIZE)
all_frames = []
total_steps_per_episode = []

for episode_num in range(NUM_EPISODES):
    agent.reset()
    frames, steps = agent.run_episode()
    all_frames.extend(frames)
    total_steps_per_episode.append(steps)
    print(f"Episode {episode_num + 1}: Total steps taken = {steps}")

fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(all_frames[0], interpolation='nearest')

# Add grid lines
ax.set_xticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.set_yticks(np.arange(-.5, GRID_SIZE, 1), minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=1)

ax.set_xticks([])  # Remove major ticks
ax.set_yticks([])
ax.set_xlim(-0.5, GRID_SIZE - 0.5)
ax.set_ylim(GRID_SIZE - 0.5, -0.5)

def update(frame):
    im.set_array(frame)
    return [im]

ani = animation.FuncAnimation(
    fig, update, frames=all_frames,
    interval=INTERVAL_MS, blit=True
)

plt.close(fig)  # Prevent static image output

HTML(ani.to_jshtml())


Episode 1: Total steps taken = 241
Episode 2: Total steps taken = 154
Episode 3: Total steps taken = 193
